In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def binomial_tree_model(S0, K, T, r, sigma, n=100):
    dt = T / n
    u = math.exp(sigma * math.sqrt(dt))
    d = math.exp(-sigma * math.sqrt(dt))
    p = (math.exp(r * dt) - d) / (u - d)
    beta = math.exp(-r * dt)
    
    # Stock prices at maturity
    stock_prices = [S0 * (u ** (n - i)) * (d ** i) for i in range(n + 1)]
    
    # Option payoffs
    call_values = [max(price - K, 0) for price in stock_prices]
    put_values = [max(K - price, 0) for price in stock_prices]
    
    # Backward induction
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            call_values[j] = beta * (p * call_values[j] + (1 - p) * call_values[j + 1])
            put_values[j] = beta * (p * put_values[j] + (1 - p) * put_values[j + 1])
    
    return call_values[0], put_values[0]


In [ ]:
def Phi(x):
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))

def black_scholes_model(S0, K, T, r, sigma):
    d1 = (math.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    call = S0 * Phi(d1) - K * math.exp(-r * T) * Phi(d2)
    put = K * math.exp(-r * T) * Phi(-d2) - S0 * Phi(-d1)
    return call, put


In [ ]:
# Input parameters
S0 = 15600
K = 15700
T = 0.25
r = 0.065
sigma = 0.1846
n = 100

# Compute results
call_bt, put_bt = binomial_tree_model(S0, K, T, r, sigma, n)
call_bs, put_bs = black_scholes_model(S0, K, T, r, sigma)

print("----------------------------------------------------")
print(" European Option Pricing Model Comparison ")
print("----------------------------------------------------")
print(f"Stock Price (S0): {S0}")
print(f"Strike Price (K): {K}")
print(f"Volatility (σ): {sigma}")
print(f"Risk-free rate (r): {r}")
print(f"Time to maturity (T): {T}")
print("----------------------------------------------------")
print(f"Binomial Model - Call: {call_bt:.4f} | Put: {put_bt:.4f}")
print(f"Black-Scholes - Call: {call_bs:.4f} | Put: {put_bs:.4f}")
print("----------------------------------------------------")
print(f"Difference (Call): {abs(call_bt - call_bs):.6f}")
print(f"Difference (Put) : {abs(put_bt - put_bs):.6f}")
print("----------------------------------------------------")


In [ ]:
volatilities = np.linspace(0.05, 0.5, 20)
binomial_calls = []
black_scholes_calls = []

for v in volatilities:
    c_bt, _ = binomial_tree_model(S0, K, T, r, v, n)
    c_bs, _ = black_scholes_model(S0, K, T, r, v)
    binomial_calls.append(c_bt)
    black_scholes_calls.append(c_bs)

plt.figure(figsize=(8,5))
plt.plot(volatilities, binomial_calls, 'o-', label='Binomial Tree Model')
plt.plot(volatilities, black_scholes_calls, 's-', label='Black-Scholes Model')
plt.title("European Call Option Price vs Volatility")
plt.xlabel("Volatility (σ)")
plt.ylabel("Call Option Price")
plt.legend()
plt.grid(True)
plt.show()
